# 1. 数据预处理

## 1.1 训练数据集预处理

In [234]:
import pandas as pd

data = pd.read_csv('./datasets/todo_train.csv')
data.head(5)

,label,text
0,1,小张，你明天需要把产品方案输出，然后给小王。小王后天把交互做一下，大后天评审。
1,1,我明天应该可以把这个文档搞定。
2,1,统计异常数据那么简单的事情，怎么可能要3天时间呢！今天下班前，务必把这个事情搞定！
3,1,李总说的这个想法，一定要在8月10日之前形成一个成熟的方案。
4,1,走廊里的空箱子太多了，小张你明天把它们全部处理掉。


In [235]:
for i in range(len(data['label'])):
    data['label'][i] = '__label__' + str(data['label'][i])

In [236]:
import jieba
jieba.enable_parallel()
data['text_division'] = data['text'].apply(lambda i:jieba.cut(i))
data['text_division'] = [' '.join(i) for i in data['text_division']]

In [237]:
data.head()

,label,text,text_division
0,__label__1,小张，你明天需要把产品方案输出，然后给小王。小王后天把交互做一下，大后天评审。,小张 ， 你 明天 需要 把 产品 方案 输出 ， 然后 给 小王 。 小 王后 天 把 交...
1,__label__1,我明天应该可以把这个文档搞定。,我 明天 应该 可以 把 这个 文档 搞定 。
2,__label__1,统计异常数据那么简单的事情，怎么可能要3天时间呢！今天下班前，务必把这个事情搞定！,统计 异常 数据 那么 简单 的 事情 ， 怎么 可能 要 3 天 时间 呢 ！ 今天 下班...
3,__label__1,李总说的这个想法，一定要在8月10日之前形成一个成熟的方案。,李总说 的 这个 想法 ， 一定 要 在 8 月 10 日 之前 形成 一个 成熟 的 方案 。
4,__label__1,走廊里的空箱子太多了，小张你明天把它们全部处理掉。,走廊 里 的 空箱子 太多 了 ， 小张 你 明天 把 它们 全部 处理 掉 。


In [238]:
with open('./datasets/todo.train','w',encoding='utf-8') as fp:
    for i in range(len(data['label'])):
        line = data['label'][i]+' , '+data['text_division'][i]
        fp.write(line)
        fp.write('\n')

## 1.2 测试数据集预处理

In [239]:
testData = pd.read_csv('./datasets/todo_test.csv',header=None)
testData.head()

,0,1
0,1,小张，你明天需要把产品方案输出，然后给小王。小王后天把交互做一下，大后天评审。
1,1,投标方案很重要，这周五定要写好，不然会来不及。
2,1,个人中心这个界面优化一下有那么难吗？不要找那么多接口，我就问你，8月底之前你能不能做完！
3,1,工程样品验收一定要尽快完成。你说的8月底太迟了，这个事情必须在8月中旬搞定，没得商量，就那么定了。
4,1,产品方案不急，明天或者后天给到我都行。


In [240]:
testData.columns = ['label','text']

In [241]:
testData.head()

,label,text
0,1,小张，你明天需要把产品方案输出，然后给小王。小王后天把交互做一下，大后天评审。
1,1,投标方案很重要，这周五定要写好，不然会来不及。
2,1,个人中心这个界面优化一下有那么难吗？不要找那么多接口，我就问你，8月底之前你能不能做完！
3,1,工程样品验收一定要尽快完成。你说的8月底太迟了，这个事情必须在8月中旬搞定，没得商量，就那么定了。
4,1,产品方案不急，明天或者后天给到我都行。


In [242]:
for i in range(len(testData['label'])):
    testData['label'][i] = '__label__'+str(testData['label'][i])

In [243]:
testData.head()

,label,text
0,__label__1,小张，你明天需要把产品方案输出，然后给小王。小王后天把交互做一下，大后天评审。
1,__label__1,投标方案很重要，这周五定要写好，不然会来不及。
2,__label__1,个人中心这个界面优化一下有那么难吗？不要找那么多接口，我就问你，8月底之前你能不能做完！
3,__label__1,工程样品验收一定要尽快完成。你说的8月底太迟了，这个事情必须在8月中旬搞定，没得商量，就那么定了。
4,__label__1,产品方案不急，明天或者后天给到我都行。


In [244]:
testData['text_division'] = testData['text'].apply(lambda i: jieba.cut(i))
testData['text_division'] = [' '.join(i) for i in testData['text_division']]

In [245]:
testData.head()

,label,text,text_division
0,__label__1,小张，你明天需要把产品方案输出，然后给小王。小王后天把交互做一下，大后天评审。,小张 ， 你 明天 需要 把 产品 方案 输出 ， 然后 给 小王 。 小 王后 天 把 交...
1,__label__1,投标方案很重要，这周五定要写好，不然会来不及。,投标 方案 很 重要 ， 这 周五 定要 写 好 ， 不然 会 来不及 。
2,__label__1,个人中心这个界面优化一下有那么难吗？不要找那么多接口，我就问你，8月底之前你能不能做完！,个人 中心 这个 界面 优化 一下 有 那么 难 吗 ？ 不要 找 那么 多 接口 ， 我 ...
3,__label__1,工程样品验收一定要尽快完成。你说的8月底太迟了，这个事情必须在8月中旬搞定，没得商量，就那么定了。,工程 样品 验收 一定 要 尽快 完成 。 你 说 的 8 月底 太迟 了 ， 这个 事情 ...
4,__label__1,产品方案不急，明天或者后天给到我都行。,产品 方案 不 急 ， 明天 或者 后天 给 到 我 都行 。


In [247]:
with open('./datasets/todo.test','w',encoding='utf-8') as fp:
    for i in range(len(testData['label'])):
        line = testData['label'][i] + ' , ' + testData['text_division'][i]
        fp.write(line)
        fp.write('\n')

# 2. 建立预测模型

## 2.1 训练数据传入训练模型

In [249]:
import fasttext.FastText as fasttext
# model = fasttext.train_supervised(input="cooking.train", lr=0.5, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='ova')
# changing the number of epochs (using the option -epoch, standard range [5 - 50]) ;
# changing the learning rate (using the option -lr, standard range [0.1 - 1.0]) ;
# using word n-grams (using the option -wordNgrams, standard range [1 - 5]).
model = fasttext.train_supervised(input='./datasets/todo.train',lr = 0.5,epoch = 15, wordNgrams=3)

## 2.2 保存模型

In [250]:
 model.save_model('todo_fastText.bin')

## 2.3 导入模型

In [251]:
model = fasttext.load_model('todo_fastText.bin')

## 2.4 验证测试

In [252]:
result = model.test('./datasets/todo.test')
print('Test set contains %d pieces of data.' %result[0])
print('Rrecision is %.2f.' %result[1])
print('Recall is %.2f.' %result[2])

Test set contains 50 pieces of data.
Rrecision is 0.96.
Recall is 0.96.


In [253]:
# predict several text together
model.predict(['你 这 两天 帮 我订 一张 8 月 20 号 去 成都 的 机票 。','老李 明天 就 会 把 方案 交给 你 验收 。'])

([['__label__1'], ['__label__1']],
 [array([0.5029745], dtype=float32), array([0.5093326], dtype=float32)])

In [254]:
model.predict('明天把文档整理好给我')

(('__label__0',), array([0.51889998]))

In [255]:
model.predict('这个 自动 预警 机制 下 周五 能 做好 吧 ， 到时候 可以 测试 一下 。')

(('__label__1',), array([0.50517923]))

In [256]:
model.predict('小 李 你 去 帮 我 做 一个 介绍 产品 的 ppt ， 主要 针对 商务 、 市场 那边 做 培训 用 的 。 最迟在 20 号 要 给。')

(('__label__1',), array([0.50086963]))